#  Chains Exercise 

## TASK:
Create function below that takes in a string input Customer Support email that could be written in any language. The function will then detect the language, translate the email, and provide a summary.

Fill out the function below using a Sequential Chain, the function should do the following:

1. Detect the language the email is written in
2. Translate the email from detected language to English
3. Return a summary of the translated email

Note: The Function should return a dictionary that contains all three of these outputs!

In [1]:
%pip install langchain
%pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 10.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.29.165 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[not

In [2]:
!pip install -q transformers accelerate langchain


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
from huggingface_hub import login
login()hugg

In [11]:
import langchain
from langchain.prompts import (
    ChatPromptTemplate, 
    PromptTemplate, 
    SystemMessagePromptTemplate, 
    AIMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    FewShotChatMessagePromptTemplate
)
from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import SequentialChain, LLMChain
from langchain_core.runnables import RunnablePassthrough

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSequenceClassification
import transformers
import torch

In [5]:
with open('api_key.txt', 'r') as file:
    api_key = file.read()

In [6]:
with open('spanish_customer_email.txt') as f:
    email = f.read()

## Detect language by BERT classifier

In [7]:
tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
model = AutoModelForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")
pipe = pipeline("text-classification", model=model,tokenizer=tokenizer, truncation=True )

In [9]:
language_detected = pipe(email)
language_detected[0]

{'label': 'es', 'score': 0.9903419017791748}

Correctly classified as Spanish with 99% probability

## Translation using Seq2SeqLM transformer

In [8]:
# %pip install sacremoses
tokenizer2 = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
model2 = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
translator = pipeline("translation", model=model2,tokenizer=tokenizer2)
input_ids = tokenizer2(email, max_length=512, truncation=True, return_tensors='pt').input_ids

/opt/python/envs/default/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/opt/python/envs/default/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
outputs = model2.generate(input_ids)

In [11]:
translated_text = tokenizer2.decode(outputs[0])
translated_text

'<pad> Subject: Low Quality Report of Cyncnic T Problems - Functionality of the SAAS Panel Estimated Support Team to the Client, I hope this message finds them well. I write them to inform you about a technical problem I found while using your SAAS panel product. As a loyal client, I appreciate the value that your product brings to my business, but I now face a challenge or requires your experience. I like to describe it in detail the problem I am experiencing: 1. Problem I am afraid: When I start out on the SAAS panel, I noticed that the greys and the tables in the main panel are not working properly. The data points appear to be distorted and some elements are supersy, which make it difficult to interpret the n-formation of the n-formation.</s>'

Model is limited by 512 tokens, truncation helped only a little bit. So only a prt of email is translated

## Using OPENAI
Langchain cannot process "text-classification" and "translation". Open source Zephyr LLM is not able to translate the text. Therefore I am forced to use OpenAI, which can translate, classify and summarize. And make a chain of it.

In [7]:
%pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate

In [26]:
with open('spanish_customer_email.txt') as f:
    email = f.read()

In [3]:
with open('open_ai_api_key.txt', 'r') as file:
    openai_api_key = file.read()

In [9]:
chat = ChatOpenAI(temperature=0.1,
                  openai_api_key=openai_api_key)

In [27]:
class Emailtranslator(BaseModel):
    language: str = Field(description="Language")
    translation: str = Field(description="Translation")
    summarization: str = Field(description="Summary")

In [28]:
parser = PydanticOutputParser(pydantic_object=Emailtranslator)

In [29]:
template_text = """In what language is this text written?\n
    Translate the text to english:\n
    Make a summarization from the english translation.\n{format_instructions}\n{text}"""

prompt = PromptTemplate(template=template_text,
                        input_variables=["text"],
                        partial_variables={"format_instructions": parser.get_format_instructions()}
)
chain =({'text':RunnablePassthrough()} | prompt |chat | parser)

In [30]:
result = chain.invoke({email})

In [45]:
print(result)

language='Spanish' translation="Subject: Technical Issue Report - SAAS Panel Functionality\n\nDear Customer Support Team,\n\nI hope this message finds you well. I am writing to inform you about a technical issue I have encountered while using your SAAS panel product. As a loyal customer, I appreciate the value your product brings to my business, but I am currently facing a challenge that requires your expertise.\n\nI would like to describe in detail the problem I am experiencing:\n\n1. Graphic Issue: Upon logging into the SAAS panel, I have noticed that the graphs and tables on the main panel page are not rendering correctly. Data points appear distorted and some elements overlap, making it difficult to accurately interpret the information.\n\n2. Export Function Failure: Additionally, I have been unable to export reports and data from the panel. Every time I try to export a report in CSV or PDF format, I receive an error message indicating that the export has failed. This functionality

In [44]:
print(result.language)
print(result.translation)
print(result.summarization)

Spanish
Subject: Technical Issue Report - SAAS Panel Functionality

Dear Customer Support Team,

I hope this message finds you well. I am writing to inform you about a technical issue I have encountered while using your SAAS panel product. As a loyal customer, I appreciate the value your product brings to my business, but I am currently facing a challenge that requires your expertise.

I would like to describe in detail the problem I am experiencing:

1. Graphic Issue: Upon logging into the SAAS panel, I have noticed that the graphs and tables on the main panel page are not rendering correctly. Data points appear distorted and some elements overlap, making it difficult to accurately interpret the information.

2. Export Function Failure: Additionally, I have been unable to export reports and data from the panel. Every time I try to export a report in CSV or PDF format, I receive an error message indicating that the export has failed. This functionality is crucial for sharing informatio